In [1]:
from pyspark.sql import SparkSession
from operator import add
import re

spark_session = SparkSession.builder\
        .master("spark://192.168.2.250:7077") \
        .appName("A3_AN_Spark2")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 8)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/21 22:01:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/21 22:01:12 WARN Utils: Service 'sparkDriver' could not bind on port 9999. Attempting port 10000.
24/02/21 22:01:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/21 22:01:13 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.


In [2]:
CSV = spark_session.read.csv('hdfs://192.168.2.250:9000/parking-citations.csv')
CSV.show()

24/02/21 22:01:14 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/02/21 22:01:14 INFO SharedState: Warehouse path is 'file:/home/ubuntu/DE1-spark/spark-warehouse'.
24/02/21 22:01:15 INFO InMemoryFileIndex: It took 58 ms to list leaf files for 1 paths.
24/02/21 22:01:15 INFO InMemoryFileIndex: It took 6 ms to list leaf files for 1 paths.
24/02/21 22:01:16 INFO FileSourceStrategy: Pushed Filters: 
24/02/21 22:01:16 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
24/02/21 22:01:17 INFO CodeGenerator: Code generated in 150.574661 ms
24/02/21 22:01:17 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.4 KiB, free 434.2 MiB)
24/02/21 22:01:17 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.5 KiB, free 434.2 MiB)
24/02/21 22:01:17 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-170-de1:10006 (size: 34.5 Ki

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|          _c0|                 _c1|       _c2|     _c3|        _c4|           _c5|              _c6| _c7| _c8|       _c9| _c10|                _c11| _c12|  _c13|          _c14|                _c15|       _c16|     _c17|     _c18|              _c19|             _c20|                _c21|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|     

24/02/21 22:01:22 INFO BlockManagerInfo: Added broadcast_3_piece0 in memory on 192.168.2.253:10005 (size: 34.4 KiB, free: 434.3 MiB)
24/02/21 22:01:22 INFO TaskSetManager: Finished task 0.0 in stage 1.0 (TID 1) in 306 ms on 192.168.2.253 (executor 0) (1/1)
24/02/21 22:01:22 INFO TaskSchedulerImpl: Removed TaskSet 1.0, whose tasks have all completed, from pool 
24/02/21 22:01:22 INFO DAGScheduler: ResultStage 1 (showString at NativeMethodAccessorImpl.java:0) finished in 0.320 s
24/02/21 22:01:22 INFO DAGScheduler: Job 1 is finished. Cancelling potential speculative or zombie tasks for this job
24/02/21 22:01:22 INFO TaskSchedulerImpl: Killing all running tasks in stage 1: Stage finished
24/02/21 22:01:22 INFO DAGScheduler: Job 1 finished: showString at NativeMethodAccessorImpl.java:0, took 0.325018 s
24/02/21 22:01:22 INFO CodeGenerator: Code generated in 27.794633 ms


In [3]:
CSV.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)



In [4]:
CSV.count()

24/02/21 22:01:22 INFO FileSourceStrategy: Pushed Filters: 
24/02/21 22:01:22 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/21 22:01:22 INFO CodeGenerator: Code generated in 9.10782 ms
24/02/21 22:01:22 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 200.3 KiB, free 433.7 MiB)
24/02/21 22:01:22 INFO BlockManagerInfo: Removed broadcast_4_piece0 on host-192-168-2-170-de1:10006 in memory (size: 9.2 KiB, free: 434.3 MiB)
24/02/21 22:01:22 INFO BlockManagerInfo: Removed broadcast_4_piece0 on 192.168.2.253:10005 in memory (size: 9.2 KiB, free: 434.3 MiB)
24/02/21 22:01:22 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 433.7 MiB)
24/02/21 22:01:22 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on host-192-168-2-170-de1:10006 (size: 34.4 KiB, free: 434.3 MiB)
24/02/21 22:01:22 INFO SparkContext: Created broadcast 5 from count at NativeMethodAccessorImpl.java:0
24/02/21 22:01:22 INFO FileSourceSc

13077725

In [5]:
rdd = CSV.rdd
numParts = rdd.getNumPartitions()
print(numParts)

16


24/02/21 22:01:25 INFO FileSourceStrategy: Pushed Filters: 
24/02/21 22:01:25 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/21 22:01:25 INFO StandaloneSchedulerBackend$StandaloneDriverEndpoint: Registered executor NettyRpcEndpointRef(spark-client://Executor) (192.168.2.249:57442) with ID 1,  ResourceProfileId 0
24/02/21 22:01:25 INFO ExecutorMonitor: New executor 1 has registered (new total is 2)
24/02/21 22:01:25 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 200.3 KiB, free 433.7 MiB)
24/02/21 22:01:25 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 433.7 MiB)
24/02/21 22:01:25 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on host-192-168-2-170-de1:10006 (size: 34.4 KiB, free: 434.3 MiB)
24/02/21 22:01:25 INFO SparkContext: Created broadcast 8 from javaToPython at NativeMethodAccessorImpl.java:0
24/02/21 22:01:25 INFO FileSourceScanExec: Planning scan with bin packing, max size: 12771

In [16]:
dropCols = ["VIN", "Latitude", "Longitude"]
CSV.drop("VIN", "Latitude", "Longitude")
# This should work according to documentation and StackOverflow, however it does not

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string]

In [18]:
CSV.show()

+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|          _c0|                 _c1|       _c2|     _c3|        _c4|           _c5|              _c6| _c7| _c8|       _c9| _c10|                _c11| _c12|  _c13|          _c14|                _c15|       _c16|     _c17|     _c18|              _c19|             _c20|                _c21|
+-------------+--------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+--------------------+-----------+---------+---------+------------------+-----------------+--------------------+
|Ticket number|          Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|     

24/02/21 22:07:44 INFO FileSourceStrategy: Pushed Filters: 
24/02/21 22:07:44 INFO FileSourceStrategy: Post-Scan Filters: 
24/02/21 22:07:44 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 200.3 KiB, free 433.5 MiB)
24/02/21 22:07:44 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 34.4 KiB, free 433.4 MiB)
24/02/21 22:07:44 INFO BlockManagerInfo: Added broadcast_21_piece0 in memory on host-192-168-2-170-de1:10006 (size: 34.4 KiB, free: 434.3 MiB)
24/02/21 22:07:44 INFO SparkContext: Created broadcast 21 from showString at NativeMethodAccessorImpl.java:0
24/02/21 22:07:44 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/02/21 22:07:44 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/02/21 22:07:44 INFO DAGScheduler: Got job 10 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions